# LIMPIEZA DE DATOS

In [15]:
import pandas as pd
import numpy as np
from datetime import datetime
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [16]:
df_clean = pd.read_csv('../data/hr_raw_data_final.csv', index_col=0)

### Homogeneizamos los datos de 'AGE'

In [17]:
def age(year):
    """
    Calcula la edad actual a partir del año de nacimiento.

    Esta función toma un año de nacimiento como entrada y devuelve la edad 
    correspondiente restando el año de nacimiento del año actual.

    Parámetros:
    year (int): Año de nacimiento.

    Retorna:
    int: Edad calculada.

    Nota:
    - Se utiliza la biblioteca `datetime` para obtener el año actual.
    - Se asume que el año de nacimiento está correctamente registrado como un número entero.
    """
    age_def = (datetime.now().year) - year
    return age_def

df_clean["age"] = df_clean["datebirth"].apply(age)

df_clean.sample(5)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
1062,46,No,travel_frequently,1032.487286,Research & Development,2,1,NaN,1,1063,2,0,NaN,3,3,ManufacTURInG DIRECtOR,3,Single,"8339,32$","21682,23$",5,Y,NaN,18,NaN,2,NaN,0,"16,0",2,"2,0",2,NaN,2,2,"8339,32$",1979,NaN,ManufacTURInG DIRECtOR - Research & Develop...,NaN,True
1404,35,No,NaN,659.079365,NaN,8,5,NaN,1,1405,4,0,NaN,3,2,HeALThCARE rEPRESEnTATIvE,3,NaN,"5323,33$","13840,67$",2,Y,NaN,17,"3,0",1,NaN,0,"14,0",6,"3,0",0,NaN,0,0,"5323,33$",1990,NaN,NaN,NaN,Yes
196,32,No,travel_rarely,556.256661,NaN,28,2,NaN,1,197,4,1,NaN,3,2,resEarcH sCiEnTISt,4,Married,"4492,84$","11681,39$",1,NaN,No,13,"3,0",4,Part Time,2,"11,0",2,"3,0",10,NaN,1,9,"4492,84$",1993,"53914,11$",NaN,NaN,Yes
30,43,No,NaN,1077.865079,Sales,-37,1,NaN,1,31,37,1,NaN,3,3,Sales ExECuTIVe,3,Divorced,"8705,83$","22635,17$",0,NaN,NaN,13,"3,0",4,Full Time,1,NaN,3,"4,0",22,NaN,13,5,"8705,83$",1982,"104470,00$",Sales ExECuTIVe - Sales,NaN,Yes
202,52,No,NaN,290.035510,NaN,28,1,Medical,1,203,4,0,NaN,4,1,LAboRaTorY teChniCiaN,3,Married,"2342,59$","6090,75$",1,Y,Yes,11,NaN,3,NaN,3,"20,0",3,"3,0",20,NaN,3,8,"2342,59$",1973,NaN,NaN,NaN,Yes


### Reemplazamos comas por puntos y eliminamos $ 

In [ ]:
def replace_dot(cadena):
    """
    Convierte valores de tipo cadena en números flotantes, reemplazando separadores y eliminando símbolos.

    Esta función procesa valores de texto que representan cantidades monetarias o numéricas 
    con comas como separadores decimales y el símbolo de dólar al final. Realiza las siguientes modificaciones:
    - Reemplaza comas (",") por puntos (".") para asegurar un formato numérico válido.
    - Elimina el símbolo de dólar ("$").
    - Convierte el resultado a tipo `float`.

    Parámetros:
    cadena (str): Cadena de texto que representa un valor numérico.

    Retorna:
    float: Número en formato decimal si la conversión es exitosa.
    np.nan: Valor NaN (Not a Number) si la conversión falla.

    Nota:
    - Se recomienda usar esta función con columnas de un DataFrame que contengan valores monetarios o numéricos 
      almacenados como cadenas de texto.
    - Se maneja cualquier error en la conversión retornando `np.nan`.

    Ejemplo:
    >>> replace_dot("1,234.56$")
    1234.56

    >>> replace_dot("invalid_value")
    nan
    """
    try:
        # Reemplazar las comas por puntos en la cadena
        return float(cadena.replace(",", ".").replace("$",""))
    
    except:
        # Si ocurre algún error (por ejemplo, si el argumento no es una cadena),
        # devolver np.nan (valor Not a Number) para indicar un valor inválido o no disponible.
        return np.nan

lista_columnas=["monthlyincome", "monthlyrate", "performancerating", "totalworkingyears", "worklifebalance", "sameasmonthlyincome", "salary"]

for columna in lista_columnas:
     df_clean[columna] = df_clean[columna].apply(replace_dot)

df_clean.head(5)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
0,53,No,NaN,2015.722222,NaN,6,3,NaN,1,1,1,0,NaN,3,5,resEArch DIREcToR,3,NaN,16280.83,42330.17,7,Y,No,13,3.0,3,Full Time,0,NaN,5,3.0,20,NaN,15,15,16280.83,1972,195370.0,NaN,NaN,Yes
1,54,No,NaN,2063.388889,NaN,1,4,Life Sciences,1,2,3,0,NaN,2,5,ManAGeR,3,NaN,NaN,43331.17,0,NaN,NaN,14,3.0,1,NaN,1,34.0,5,3.0,33,NaN,11,9,NaN,1971,199990.0,NaN,NaN,1
2,44,No,travel_rarely,1984.253968,Research & Development,4,2,Technical Degree,1,3,3,0,NaN,3,5,ManaGER,4,Married,NaN,41669.33,1,NaN,No,11,3.0,4,NaN,0,22.0,3,NaN,22,NaN,11,15,NaN,1981,192320.0,ManaGER - Research & Development,NaN,1
3,49,No,travel_rarely,1771.404762,NaN,2,4,Medical,1,4,1,1,NaN,3,4,ReseArCH DIrECtOr,3,Married,14307.50,37199.50,3,Y,NaN,19,3.0,2,Full Time,2,NaN,2,NaN,20,NaN,5,6,14307.50,1976,171690.0,NaN,NaN,False
4,48,No,NaN,1582.771346,NaN,3,3,Technical Degree,1,5,1,1,NaN,4,4,sAleS EXECUtIve,1,Divorced,12783.92,33238.20,2,Y,No,12,3.0,4,NaN,1,NaN,5,3.0,19,NaN,2,8,12783.92,1977,NaN,NaN,NaN,0


### redondear valores de tipo float con muchos decimales, a 2 decimales

In [19]:
df_clean["dailyrate"] = round(df_clean["dailyrate"], 2)
df_clean["hourlyrate"] = round(df_clean["hourlyrate"], 2)
df_clean.head(2)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
0,53,No,NaN,2015.72,NaN,6,3,NaN,1,1,1,0,NaN,3,5,resEArch DIREcToR,3,NaN,16280.83,42330.17,7,Y,No,13,3.0,3,Full Time,0,NaN,5,3.0,20,NaN,15,15,16280.83,1972,195370.0,NaN,NaN,Yes
1,54,No,NaN,2063.39,NaN,1,4,Life Sciences,1,2,3,0,NaN,2,5,ManAGeR,3,NaN,NaN,43331.17,0,NaN,NaN,14,3.0,1,NaN,1,34.0,5,3.0,33,NaN,11,9,NaN,1971,199990.0,NaN,NaN,1


In [20]:
df_clean['dailyrate'] = df_clean.apply(lambda row: np.round(row['dailyrate'], 2), axis = 1)

In [21]:
def eliminar_columnas(df, nombre_columna):
    df_drop = df.drop(nombre_columna, axis=1, inplace=True) # hay que asignarle una variable para que lo guarde. Comprobado que funciona, cambiamos el inplace por True.
    print(f'La columna {nombre_columna} se ha eliminado correctamente.')
    return df_drop.head(1)

### Homogeneizamos 'REMOTEWORK'

In [22]:
diccionario_remote = {"1": "Yes", "0": "No", "Yes": "Yes", "True": "Yes", "False": "No"}
df_clean["remotework"] = df_clean["remotework"].map(diccionario_remote)

df_clean.head(2)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
0,53,No,NaN,2015.72,NaN,6,3,NaN,1,1,1,0,NaN,3,5,resEArch DIREcToR,3,NaN,16280.83,42330.17,7,Y,No,13,3.0,3,Full Time,0,NaN,5,3.0,20,NaN,15,15,16280.83,1972,195370.0,NaN,NaN,Yes
1,54,No,NaN,2063.39,NaN,1,4,Life Sciences,1,2,3,0,NaN,2,5,ManAGeR,3,NaN,NaN,43331.17,0,NaN,NaN,14,3.0,1,NaN,1,34.0,5,3.0,33,NaN,11,9,NaN,1971,199990.0,NaN,NaN,Yes


### Cambiamos los datos a minúsculas

In [23]:
def minusculas(cadena): 
    """
    Convierte una cadena de texto a minúsculas.

    Esta función toma una cadena y devuelve la misma en minúsculas. 
    Si el argumento no es una cadena válida, devuelve "no data".

    Parámetros:
    cadena (str): Cadena de texto a transformar.

    Retorna:
    str: La cadena en minúsculas si la conversión es exitosa.
    str: "no data" si la entrada no es una cadena válida.

    Ejemplo:
    >>> minusculas("Marketing")
    'marketing'

    >>> minusculas(123)
    'no data'

    Nota:
    - Se usa un bloque `try-except` para manejar errores en caso de que 
      la entrada no sea un string.
    - Esta función es útil para normalizar valores categóricos en un DataFrame.
    """
    try:
        return cadena.lower()
    except:
        return "no data"

lista_columnas=["department", "educationfield", "jobrole", "roledepartament"]
for col in lista_columnas:
    df_clean[col] = df_clean[col].apply(minusculas)

### Normalizamos los valores de "ENVIRONMENTSATISFACTION"

In [24]:
def clean_satisfaction(valoracion):
    """
    Normaliza valores de satisfacción en una escala de 1 a 10.

    Esta función ajusta las valoraciones que son mayores o iguales a 10 
    dividiendo por 10 y extrayendo la parte entera. Si el valor ya está 
    en el rango esperado (menor a 10), se mantiene sin cambios.

    Parámetros:
    valoracion (int o float): Valor numérico de satisfacción.

    Retorna:
    int: Valor normalizado en la escala de 1 a 10.
    np.nan: Si la entrada no es un número válido.

    Ejemplo:
    >>> clean_satisfaction(35)
    3

    >>> clean_satisfaction(7)
    7

    >>> clean_satisfaction("NaN")
    nan

    Nota:
    - Se usa `try-except` para manejar errores y devolver `np.nan` en caso de datos no numéricos.
    - Se recomienda importar `numpy` (`import numpy as np`) antes de usar la función.
    """
    try: 
        if valoracion >= 10:
            return int(str(valoracion/10).split('.')[0])
        else: 
            return valoracion
    except:
            return np.nan


df_clean["environmentsatisfaction"] = df_clean["environmentsatisfaction"].apply(clean_satisfaction)

### GENDER

In [25]:
dict_gender = {0: "M", 1: "F"}

df_clean["gender"] = df_clean["gender"].map(dict_gender)

### Hacemos una función para borrar columnas

In [26]:
def clean (dataframe,col_data):
    """
    Elimina una columna específica de un DataFrame.

    Esta función elimina una columna del DataFrame especificado utilizando `drop()`. 
    La operación se realiza en el lugar (`inplace=True`), lo que significa que el 
    DataFrame original se modifica y la función devuelve `None`.

    Parámetros:
    dataframe (pd.DataFrame): DataFrame del cual se eliminará la columna.
    col_data (str): Nombre de la columna a eliminar.

    Retorna:
    None: La modificación es aplicada directamente al DataFrame.

    Ejemplo:
    >>> df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
    >>> clean(df, "A")
    >>> print(df)
       B
    0  4
    1  5
    2  6

    Nota:
    - `inplace=True` modifica el DataFrame original y no devuelve un nuevo DataFrame.
    - Si la columna no existe, Pandas generará un error `KeyError`.
    - Se recomienda verificar la existencia de la columna antes de llamar a esta función.
    """
    return dataframe.drop([col_data],axis=1, inplace= True)


### Eliminamos las columnas "sameasmonthlyincome", "over18", "numberchildren", "employeecount" y "yearsincurrentrole", por considerarlas duplicadas, redundantes o que no aportan al estudio

In [27]:
lista_drop= ["sameasmonthlyincome", "over18", "numberchildren", "employeecount", "yearsincurrentrole" ]

for col in lista_drop:
    clean(df_clean, col)


In [28]:
df_clean[["yearssincelastpromotion", "yearsatcompany"]].sample(10)

,yearssincelastpromotion,yearsatcompany
1614,1,9
1037,0,4
1243,0,1
997,0,6
1372,0,1
219,0,10
816,1,5
1597,14,24
72,2,15
1617,10,15


In [29]:
df_clean[["joblevel","jobrole", "roledepartament","department"]].sample(10)

,joblevel,jobrole,roledepartament,department
1017,1,laboratory technician,no data,no data
430,2,sales executive,no data,no data
1344,3,sales executive,sales executive - sales,sales
27,2,manufacturing director,no data,no data
1638,2,sales executive,no data,no data
656,2,sales executive,no data,no data
1677,2,research scientist,no data,no data
167,2,sales executive,sales executive - sales,sales
1649,3,manufacturing director,no data,no data
710,1,research scientist,no data,no data


### Guardamos el CSV resultante hasta ahora para tener otro punto de partida

In [30]:
df_clean.to_csv("hr_raw_data_clean.csv")

### Comprobamos si hay valores negativos en alguna columna más, aparte de 'distancefromhome'

In [31]:
'''Sacando columnas numéricas para ver si hay alguna columna más con negativos aparte del distancefromhome'''
df_numericas = df_clean.select_dtypes(include=['number'])

In [ ]:
''' ¿Cuántas veces está el - en cada columna numérica? Vemos que solo está presente en distancefromhome'''
for col in df_numericas.columns:
    """
Este código recorre todas las columnas de un DataFrame numérico y cuenta cuántos valores contienen 
el carácter '-' en cada columna.

Funcionamiento:
- Convierte los valores de cada columna a tipo `str`.
- Usa `.str.contains('-', regex=False)` para detectar la presencia del carácter '-'.
- Suma el número de veces que aparece el carácter en la columna.
- Imprime el resultado indicando la cantidad de valores con '-' en cada columna.

Parámetros:
- `df_numericas` (pd.DataFrame): DataFrame que contiene columnas numéricas.

Salida esperada:
- Mensajes en la consola indicando cuántos valores con '-' existen en cada columna.

Ejemplo de salida:
- " - está presente en salary 15 veces."
- " - está presente en monthlyincome 3 veces."

Nota:
- Este método es útil para detectar valores negativos o errores en los datos numéricos.
- Si la columna contiene valores NaN, estos no afectarán el conteo.
"""
    num_menos_col = df_numericas[col].astype(str).str.contains('-', regex=False).sum()
    print(f' - está presente en {col} {num_menos_col} veces.')

 - está presente en age 0 veces.
 - está presente en dailyrate 0 veces.
 - está presente en distancefromhome 202 veces.
 - está presente en education 0 veces.
 - está presente en employeenumber 0 veces.
 - está presente en environmentsatisfaction 0 veces.
 - está presente en hourlyrate 0 veces.
 - está presente en jobinvolvement 0 veces.
 - está presente en joblevel 0 veces.
 - está presente en jobsatisfaction 0 veces.
 - está presente en monthlyincome 0 veces.
 - está presente en monthlyrate 0 veces.
 - está presente en numcompaniesworked 0 veces.
 - está presente en percentsalaryhike 0 veces.
 - está presente en performancerating 0 veces.
 - está presente en relationshipsatisfaction 0 veces.
 - está presente en stockoptionlevel 0 veces.
 - está presente en totalworkingyears 0 veces.
 - está presente en trainingtimeslastyear 0 veces.
 - está presente en worklifebalance 0 veces.
 - está presente en yearsatcompany 0 veces.
 - está presente en yearssincelastpromotion 0 veces.
 - está pre

### Convertimos los valores negativos de 'distancefromhome' en absolutos

In [ ]:
def convert_negatives_in_absolute(df, columns): 
    """
    Convierte valores negativos en positivos en las columnas especificadas de un DataFrame.

    Esta función aplica la función `abs()` a las columnas seleccionadas, convirtiendo 
    todos los valores negativos en su equivalente absoluto.

    Parámetros:
    df (pd.DataFrame): DataFrame que contiene los datos a modificar.
    columns (list): Lista de nombres de las columnas en las que se aplicará la conversión.

    Retorna:
    pd.DataFrame: DataFrame con las columnas especificadas transformadas a valores absolutos.

    Ejemplo:
    >>> import pandas as pd
    >>> data = {'A': [-10, 5, -3], 'B': [7, -2, -8], 'C': [1, 2, 3]}
    >>> df = pd.DataFrame(data)
    >>> df = convert_negatives_in_absolute(df, ['A', 'B'])
    >>> print(df)
         A  B  C
    0   10  7  1
    1    5  2  2
    2    3  8  3

    Nota:
    - La función modifica directamente el DataFrame original.
    - Si se desean conservar los datos originales, se recomienda hacer una copia antes de llamar a la función.
    """
    df[columns] = df[columns].abs()  # Aplica abs() solo a las columnas seleccionadas 
    return df # Llamar a la función, por ejemplo, para convertir las columnas 'A' y 'B'
df_clean = convert_negatives_in_absolute(df_clean, 'distancefromhome') # Mostrar el resultado print(df)
df_clean.sample(2)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,salary,roledepartament,remotework
891,26,No,NaN,556.26,no data,10,4,marketing,892,4,F,NaN,3,2,sales executive,3,Divorced,4492.84,11681.39,1,Yes,12,3.0,4,Part Time,1,NaN,2,4.0,5,0,3,1999,53914.11,no data,Yes
1577,40,No,travel_rarely,556.26,sales,7,4,marketing,1578,4,F,NaN,2,2,sales executive,4,NaN,4492.84,11681.39,0,No,20,NaN,1,Part Time,0,NaN,2,3.0,7,0,5,1985,53914.11,sales executive - sales,Yes


In [34]:
'''Comprobación de negativos'''
df_clean['distancefromhome'].astype(str).str.contains('-', regex=False).sum()

0

### Creamos la columna 'correct_department' que contendrá información correcta a partir de los datos que contienen las columnas 'jobrole', 'roledepartament' y 'department'

In [35]:
'''Creamos un diccionario con los puestos como claves y los departamentos como valores.
Función que asigna el departamento según el puesto. Si no encuentra la clave en el diccionario, devuelve el valor de la columna
roledepartament para que no se cambien los manager que sí están asignados a algunos departamentos concretos - sales, human resources, etc.'''

dic = {'healthcare representative': 'research & development',
       'sales executive': 'sales',
       'laboratory technician': 'research & development',
       'manufacturing director': 'research & development',
       'research scientist': 'research & development',
       'research director': 'research & development',
       'human resources': 'human resources',
       'sales representative': 'sales'}

def assign_departament(puesto, departamento, otra_columna):
    """
    Asigna un departamento a un puesto específico según un diccionario de mapeo.

    Parámetros:
    puesto (str): Nombre del puesto de trabajo.
    departamento (dict): Diccionario que mapea cada puesto a su departamento correspondiente.
    otra_columna (str): Valor alternativo a devolver si el puesto no está en el diccionario.

    Retorna:
    str: Nombre del departamento asignado al puesto. Si el puesto no está en el diccionario, 
         se devuelve el valor de `otra_columna`.

    Ejemplo:
    >>> dic = {'sales executive': 'sales', 'research scientist': 'research & development'}
    >>> assign_departament('sales executive', dic, 'unknown')
    'sales'

    >>> assign_departament('manager', dic, 'unknown')
    'unknown'
    """
    return departamento.get(puesto, otra_columna)

# Limpiamos la columna 'jobrole' de espacios y le hacemos un lower, ya que no estaba reconociendo las claves y es posible que hubiera algún 
# espacio en alguna celda.
df_clean['jobrole'] = df_clean['jobrole'].str.strip().str.lower()

# Asignamos el departamento correspondiente para cada puesto en la columna 'correct_department'
df_clean['correct_department'] = df_clean.apply(lambda fila: assign_departament(fila['jobrole'], dic, fila['roledepartament']), axis=1)

### Una vez unificados los datos en la columna 'correct_department', podemos prescindir de  'roledepartament' Y 'department'

In [36]:
# Utilizamos función creada previamente.

lista_drop1= ["roledepartament", "department"]

for col in lista_drop1:
    clean(df_clean, col)

### Como ya no es posible la duplicidad, vamos a cambiar el nombre de la columna 'correct_department' a 'department'

In [37]:
df_clean.rename(columns={'correct_department': 'department'}, inplace=True)

### Rellenamos los valores que faltan en la columna 'salary' multiplicando el 'monthlyincome' * 12.

In [38]:
'''Si la celda es mayor a 0, es decir tiene dato, se queda tal cual, si no, multiplicamos 'monthlyincome' * 12 para calcular salario anual.'''
df_clean['salary'] = df_clean.apply(lambda row: row['salary'] if row['salary'] > 0 else row['monthlyincome'] * 12, axis = 1)

### Rellenamos los valores que faltan en la columna 'monthlyincome' dividiendo el 'salary' / 12.

In [39]:
df_clean['monthlyincome'] = df_clean.apply(lambda row: row['monthlyincome'] if row['monthlyincome'] > 0 else row['salary'] / 12, axis = 1)

In [40]:
'''Comprobamos que el dato de 'salary', que es anual, corresponder con con su valor mensual 'monthlyincome' '''
comp = round(df_clean['salary']/12 - df_clean['monthlyincome'], 2)
comp.unique()

array([     0.  , -83242.17, -86790.33, -93142.17])

In [41]:
'''Observamos que hay valores mensuales mayores que anuales, comprobamos cuáles son'''
df_clean[['monthlyincome', 'salary', 'employeenumber']].loc[df_clean['monthlyincome'] > df_clean['salary']]

,monthlyincome,salary,employeenumber
1316,84083.0,10090.0,1317
1359,87667.0,10520.0,1360
1464,94083.0,11290.0,1465


In [42]:
''' Observamos que el error es debido a que el dato "monthlyincome" está multiplicado por 100. Cambiamos estos valores usando su índice'''
df_clean.loc[df_clean['employeenumber'].isin([1317,1360,1465]), "monthlyincome"]/100

1316    840.83
1359    876.67
1464    940.83
Name: monthlyincome, dtype: float64


### Hacemos una función para cambiar los nulos por un dato en concreto

In [43]:
def nulos(df,columna, dato): 
    df[columna] = df[columna].fillna(dato)
    return 

In [44]:
'''Cambiamos los datos nulos de la columna "businesstravel" por el dato : 'non-travel' '''
nulos(df_clean, "businesstravel", "non-travel")

In [45]:
'''Cambiamos los datos nulos de la columna "standardhours" por el dato : 'Full Time' '''
nulos(df_clean, "standardhours", "Full Time")

### Conociendo que la jornada laboral tiene 8 horas, rellenamos los nulos de 'hourlyrate' por 'dailyrate' dividido entre 8

In [46]:
df_clean['hourlyrate'] = df_clean.apply(lambda row: row['dailyrate'] / 8, axis = 1)

### Redondeamos los decimales de "monthlyincome"

In [47]:
df_clean["monthlyincome"] = round(df_clean["monthlyincome"], 2)

### Existe un error de escritura en los datos de 'maritalstatus', por lo que sustituimos estos datos por el correcto

In [48]:
def marital_status (dato):
    """
    Normaliza el estado civil en formato estándar.

    Esta función toma un valor de estado civil y realiza las siguientes operaciones:
    - Si el valor es 'Marreid', lo corrige a 'married'.
    - Para cualquier otro valor, lo convierte a minúsculas.
    - Si ocurre un error en la entrada, devuelve `np.nan`.

    Parámetros:
    dato (str): El valor que representa el estado civil.

    Retorna:
    str: El estado civil corregido (en minúsculas o 'married') o `np.nan` si ocurre un error.

    Ejemplo:
    >>> marital_status('Marreid')
    'married'

    >>> marital_status('Single')
    'single'

    >>> marital_status(None)
    nan

    Nota:
    - Se maneja un error genérico con `try-except`, por lo que cualquier entrada que cause un error devolverá `np.nan`.
    - El valor 'Marreid' parece ser un error tipográfico que se corrige a 'married'.
    """
    try:

        if dato == 'Marreid':
            return dato.rename(columns= {'Marreid': 'married'})
        else:
            return str(dato).lower()
    except:
        return np.nan 

In [49]:
df_clean['maritalstatus'] = df_clean['maritalstatus'].apply(marital_status)

## Función para rellenar valores nulos

In [70]:
def rellenar_nulos(dataframe, columna, relleno):
    # Si la columna es numérica y el relleno es una cadena, la columna debe ser convertida a tipo objeto (string).
    if isinstance(relleno, str):
        dataframe[columna] = dataframe[columna].astype("object")
    
    dataframe[columna] = dataframe[columna].fillna(relleno)  # Asignación directa sin 'inplace=True'


In [ ]:
''' Rellenamos los valores nulos de "performancerating" por el valor "not rated"'''
rellenar_nulos(df_clean, "performancerating", "not rated")

In [73]:
''' Comprobamos'''
df_clean["performancerating"].unique()

array([3.0, 4.0, 'not rated'], dtype=object)

In [72]:
''' Rellenamos los valores nulos de "maritalstatus" por el valor "unknown"'''
rellenar_nulos(df_clean, "maritalstatus", "unknown")

In [53]:
''' Comprobamos'''
df_clean["maritalstatus"].unique()

array(['nan', 'married', 'divorced', 'single', 'unknown'], dtype=object)

### En "educationfield", cambiamos los valores 'no data' por 'other' 

In [54]:
df_clean["educationfield"] = df_clean["educationfield"].replace("no data", "other")

In [74]:
'''Comprobamos'''
df_clean["educationfield"].unique()

array(['other', 'life sciences', 'technical degree', 'medical',
       'marketing', 'human resources'], dtype=object)

### En "department", cambiamos los valores 'no data' por 'general' 

In [55]:
df_clean["department"] = df_clean["department"].replace("no data", "general")

In [56]:
'''Comprobamos'''
df_clean["department"].unique()

array(['research & development', 'general',
       ' manager  -  research & development ', 'sales', 'human resources',
       ' manager  -  sales ', ' manager  -  human resources '],
      dtype=object)

In [57]:
df_clean["department"].str.strip("-")
df_clean['department'] = df_clean['department'].str.replace('manager  -', '').str.strip()

### Eliminamos duplicados

In [58]:
# Comprobamos los duplicados
df_clean.duplicated().sum()

64

In [59]:
duplicados = df_clean[df_clean.duplicated(keep=False)] 
duplicados

,age,attrition,businesstravel,dailyrate,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,salary,remotework,department
8,43,No,non-travel,1712.18,2,5,other,9,2,F,214.02250,3,4,manager,1,married,13829.17,35955.83,7,No,16,3.0,2,Full Time,1,22.0,2,3.0,18,11,8,1982,165950.00,Yes,general
60,38,No,non-travel,610.17,5,2,other,61,4,M,76.27125,3,2,laboratory technician,2,single,4928.33,12813.67,8,No,16,3.0,4,Full Time,0,16.0,3,4.0,13,3,7,1987,59140.00,Yes,research & development
75,49,No,travel_rarely,1032.49,4,3,life sciences,76,3,F,129.06125,2,3,manufacturing director,2,divorced,8339.32,21682.23,8,Yes,12,not rated,3,Part Time,1,NaN,4,3.0,22,14,10,1976,100071.84,Yes,research & development
107,31,No,travel_rarely,1032.49,21,4,life sciences,108,2,F,129.06125,4,3,manufacturing director,1,divorced,8339.32,21682.23,1,No,11,3.0,3,Full Time,1,10.0,1,3.0,10,8,8,1994,100071.84,No,research & development
111,32,No,travel_rarely,1032.49,5,3,other,112,2,F,129.06125,3,3,sales executive,4,nan,8339.32,21682.23,2,No,12,3.0,3,Part Time,1,NaN,2,3.0,10,7,4,1993,100071.84,Yes,sales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1673,45,No,non-travel,488.94,26,3,medical,824,2,F,61.11750,4,1,research scientist,3,single,3949.17,10267.83,4,NaN,12,3.0,4,Full Time,0,NaN,2,3.0,3,1,2,1980,47390.04,Yes,research & development
1674,49,No,non-travel,1973.98,26,4,other,1087,4,F,246.74750,3,5,manager,3,married,15943.72,41453.67,3,No,11,3.0,3,Full Time,1,27.0,2,3.0,5,1,0,1976,191324.62,No,general
1675,31,No,travel_rarely,290.04,15,3,other,528,3,M,36.25500,3,1,research scientist,4,nan,2342.59,6090.75,1,No,19,3.0,1,Part Time,0,6.0,1,3.0,6,1,5,1994,28111.13,No,research & development
1676,49,No,travel_rarely,1032.49,4,3,life sciences,76,3,F,129.06125,2,3,manufacturing director,2,divorced,8339.32,21682.23,8,Yes,12,not rated,3,Part Time,1,NaN,4,3.0,22,14,10,1976,100071.84,Yes,research & development


In [75]:
''' Comprobamos que todos los duplicados los son'''
son_iguales = duplicados.nunique() == 1
son_iguales.sum()

0

In [61]:
''' Borramos los duplicados'''
df_sin_duplicados = df_clean.drop_duplicates()

In [62]:
df_sin_duplicados.head()

,age,attrition,businesstravel,dailyrate,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,salary,remotework,department
0,53,No,non-travel,2015.72,6,3,other,1,1,M,251.96500,3,5,research director,3,nan,16280.83,42330.17,7,No,13,3.0,3,Full Time,0,NaN,5,3.0,20,15,15,1972,195370.00,Yes,research & development
1,54,No,non-travel,2063.39,1,4,life sciences,2,3,M,257.92375,2,5,manager,3,nan,16665.83,43331.17,0,NaN,14,3.0,1,Full Time,1,34.0,5,3.0,33,11,9,1971,199990.00,Yes,general
2,44,No,travel_rarely,1984.25,4,2,technical degree,3,3,M,248.03125,3,5,manager,4,married,16026.67,41669.33,1,No,11,3.0,4,Full Time,0,22.0,3,NaN,22,11,15,1981,192320.00,Yes,research & development
3,49,No,travel_rarely,1771.40,2,4,medical,4,1,F,221.42500,3,4,research director,3,married,14307.50,37199.50,3,NaN,19,3.0,2,Full Time,2,NaN,2,NaN,20,5,6,1976,171690.00,No,research & development
4,48,No,non-travel,1582.77,3,3,technical degree,5,1,F,197.84625,4,4,sales executive,1,divorced,12783.92,33238.20,2,No,12,3.0,4,Full Time,1,NaN,5,3.0,19,2,8,1977,153407.04,No,sales


### Ordenamos las columnas para que tengan sentido en proximidad

In [63]:
# Cambiar el orden de las columnas# Primero este paso!!!!!!
nuevo_orden = ['employeenumber','attrition','joblevel', 'jobrole','department','monthlyincome','salary','percentsalaryhike','trainingtimeslastyear','stockoptionlevel','hourlyrate','dailyrate','monthlyrate','standardhours','overtime','businesstravel','distancefromhome','remotework','environmentsatisfaction','jobinvolvement','jobsatisfaction','performancerating','relationshipsatisfaction','worklifebalance','datebirth','age','gender','maritalstatus','education', 'educationfield','numcompaniesworked','totalworkingyears','yearsatcompany','yearssincelastpromotion','yearswithcurrmanager']
df_sin_duplicados=df_sin_duplicados[nuevo_orden]
df_sin_duplicados.head(2)

,employeenumber,attrition,joblevel,jobrole,department,monthlyincome,salary,percentsalaryhike,trainingtimeslastyear,stockoptionlevel,hourlyrate,dailyrate,monthlyrate,standardhours,overtime,businesstravel,distancefromhome,remotework,environmentsatisfaction,jobinvolvement,jobsatisfaction,performancerating,relationshipsatisfaction,worklifebalance,datebirth,age,gender,maritalstatus,education,educationfield,numcompaniesworked,totalworkingyears,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager
0,1,No,5,research director,research & development,16280.83,195370.0,13,5,0,251.96500,2015.72,42330.17,Full Time,No,non-travel,6,Yes,1,3,3,3.0,3,3.0,1972,53,M,nan,3,other,7,NaN,20,15,15
1,2,No,5,manager,general,16665.83,199990.0,14,5,1,257.92375,2063.39,43331.17,Full Time,NaN,non-travel,1,Yes,3,2,3,3.0,1,3.0,1971,54,M,nan,4,life sciences,0,34.0,33,11,9


Cambiamos el nombre de las columnas para que sean mas legibles

In [64]:
# Cambiamos el nombre de las columnas
df_sin_duplicados.columns = ['employee_number','attrition','job_level', 'job_role','department','monthly_income','salary','percent_salary_hike','training_times_last_year','stock_option_level','hourly_rate','daily_rate','monthly_rate','standard_hours','overtime','business_travel','distance_from_home','remote_work','environment_satisfaction','job_involvement','job_satisfaction','performance_rating','relationship_satisfaction','work_life_balance','date_birth','age','gender','marital_status','education', 'education_field','num_companies_worked','total_working_years','years_at_company','years_since_last_promotion','years_with_curr_manager']
df_sin_duplicados.head(2)

,employee_number,attrition,job_level,job_role,department,monthly_income,salary,percent_salary_hike,training_times_last_year,stock_option_level,hourly_rate,daily_rate,monthly_rate,standard_hours,overtime,business_travel,distance_from_home,remote_work,environment_satisfaction,job_involvement,job_satisfaction,performance_rating,relationship_satisfaction,work_life_balance,date_birth,age,gender,marital_status,education,education_field,num_companies_worked,total_working_years,years_at_company,years_since_last_promotion,years_with_curr_manager
0,1,No,5,research director,research & development,16280.83,195370.0,13,5,0,251.96500,2015.72,42330.17,Full Time,No,non-travel,6,Yes,1,3,3,3.0,3,3.0,1972,53,M,nan,3,other,7,NaN,20,15,15
1,2,No,5,manager,general,16665.83,199990.0,14,5,1,257.92375,2063.39,43331.17,Full Time,NaN,non-travel,1,Yes,3,2,3,3.0,1,3.0,1971,54,M,nan,4,life sciences,0,34.0,33,11,9


### Redondeamos valores de tipo float con muchos decimales, a 2 decimales

In [65]:
df_clean["dailyrate"] = round(df_clean["dailyrate"], 2) #(1)
df_clean["hourlyrate"] = round(df_clean["hourlyrate"], 2)
df_clean.head(2)

,age,attrition,businesstravel,dailyrate,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,salary,remotework,department
0,53,No,non-travel,2015.72,6,3,other,1,1,M,251.96,3,5,research director,3,nan,16280.83,42330.17,7,No,13,3.0,3,Full Time,0,NaN,5,3.0,20,15,15,1972,195370.0,Yes,research & development
1,54,No,non-travel,2063.39,1,4,life sciences,2,3,M,257.92,2,5,manager,3,nan,16665.83,43331.17,0,NaN,14,3.0,1,Full Time,1,34.0,5,3.0,33,11,9,1971,199990.0,Yes,general


In [66]:
# Otra forma de hacer esto (1)
df_clean['dailyrate'] = df_clean.apply(lambda row: np.round(row['dailyrate'], 2), axis = 1)

### Función para eliminar columnas -- No se utiliza.


In [67]:
def eliminar_columnas(df, nombre_columna):
    '''
    Elimina una columna específica de un DataFrame y muestra un mensaje confirmando la eliminación.

    Parámetros:
    df (DataFrame): El DataFrame del que se va a eliminar la columna.
    nombre_columna (str): El nombre de la columna a eliminar.

    Retorna:
    DataFrame: El DataFrame después de eliminar la columna especificada. 
    Si la columna no existe, se mostrará un error.
    
    Ejemplo:
    df_clean = eliminar_columnas(df_clean, 'columna_a_eliminar')
    '''
    df_drop = df.drop(nombre_columna, axis=1, inplace=True) # hay que asignarle una variable para que lo guarde. Comprobado que funciona, cambiamos el inplace por True.
    print(f'La columna {nombre_columna} se ha eliminado correctamente.')
    return df_drop.head(1)

In [ ]:
'''Comprobamos cómo queda el dataframe'''
df_clean

,age,attrition,businesstravel,dailyrate,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,salary,remotework,department
0,53,No,non-travel,2015.72,6,3,other,1,1,M,251.96,3,5,research director,3,nan,16280.83,42330.17,7,No,13,3.0,3,Full Time,0,NaN,5,3.0,20,15,15,1972,195370.00,Yes,research & development
1,54,No,non-travel,2063.39,1,4,life sciences,2,3,M,257.92,2,5,manager,3,nan,16665.83,43331.17,0,NaN,14,3.0,1,Full Time,1,34.0,5,3.0,33,11,9,1971,199990.00,Yes,general
2,44,No,travel_rarely,1984.25,4,2,technical degree,3,3,M,248.03,3,5,manager,4,married,16026.67,41669.33,1,No,11,3.0,4,Full Time,0,22.0,3,NaN,22,11,15,1981,192320.00,Yes,research & development
3,49,No,travel_rarely,1771.40,2,4,medical,4,1,F,221.42,3,4,research director,3,married,14307.50,37199.50,3,NaN,19,3.0,2,Full Time,2,NaN,2,NaN,20,5,6,1976,171690.00,No,research & development
4,48,No,non-travel,1582.77,3,3,technical degree,5,1,F,197.85,4,4,sales executive,1,divorced,12783.92,33238.20,2,No,12,3.0,4,Full Time,1,NaN,5,3.0,19,2,8,1977,153407.04,No,sales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1673,45,No,non-travel,488.94,26,3,medical,824,2,F,61.12,4,1,research scientist,3,single,3949.17,10267.83,4,NaN,12,3.0,4,Full Time,0,NaN,2,3.0,3,1,2,1980,47390.04,Yes,research & development
1674,49,No,non-travel,1973.98,26,4,other,1087,4,F,246.75,3,5,manager,3,married,15943.72,41453.67,3,No,11,3.0,3,Full Time,1,27.0,2,3.0,5,1,0,1976,191324.62,No,general
1675,31,No,travel_rarely,290.04,15,3,other,528,3,M,36.26,3,1,research scientist,4,nan,2342.59,6090.75,1,No,19,3.0,1,Part Time,0,6.0,1,3.0,6,1,5,1994,28111.13,No,research & development
1676,49,No,travel_rarely,1032.49,4,3,life sciences,76,3,F,129.06,2,3,manufacturing director,2,divorced,8339.32,21682.23,8,Yes,12,not rated,3,Part Time,1,NaN,4,3.0,22,14,10,1976,100071.84,Yes,research & development


### Guardamos en CSV para tener otro punto de partida

In [69]:
df_sin_duplicados.to_csv("../data/hr_raw_data_clean.csv")